# Setup

In [ ]:
!pip install ibm_db
!pip install ibm_db_sa

import ibm_db                          # For connecting to the database
import ibm_db_dbi                      # For other APIs to connec to the database
import ibm_db_sa                       # Adapter for SQL Alchemy
import pandas as pd                    # For dataframes and data handling
from sqlalchemy import *               # For writing to the database


In [ ]:
# Service credentials
dsn_hostname = "815fa4db-dc03-4c70-869a-a9cc13f33084.bs2io90l08kqb1od8lcg.databases.appdomain.cloud"
dsn_uid = "gjd38176"
dsn_pwd = "r60hPrG8JBjitoNt"

dsn_driver = "{IBM DB2 ODBC DRIVER}"
dsn_database = "BLUDB"
dsn_port = "30367"
dsn_protocol = "TCPIP"
dsn_security = "SSL"

# Connection string
dsn = (
"DRIVER={0};"
"DATABASE={1};"
"HOSTNAME={2};"
"PORT={3};"
"PROTOCOL={4};"
"UID={5};"
"PWD={6};"
"SECURITY={7};").format(dsn_driver, dsn_database, dsn_hostname, dsn_port, dsn_protocol, dsn_uid, dsn_pwd,dsn_security)

# Connect

In [ ]:
# Establish connections
try:
    db_conn = ibm_db.connect(dsn, "", "")
    dbi_conn = ibm_db_dbi.Connection(db_conn)
    print ("Connected to database: ", dsn_database, "as user: ", dsn_uid, "on host: ", dsn_hostname)

except:
    print ("Unable to connect: ", ibm_db.conn_errormsg() ) 

In [ ]:
# SQL Alchemy connection
# Supply connection string to engine
#sqla_str= "db2+ibm_db://" + dsn_uid+ ':' + dsn_pwd + '@' + dsn_hostname + ':' + dsn_port + '/' + dsn_database
sqla_str = "ibm_db_sa://" + dsn_uid+ ':' + dsn_pwd + '@' + dsn_hostname + ':' + dsn_port + '/' + dsn_database
print(sqla_str)
try:
    #engine = create_engine("db2+ibm_db://gjd38176:r60hPrG8JBjitoNt@815fa4db-dc03-4c70-869a-a9cc13f33084.bs2io90l08kqb1od8lcg.databases.appdomain.cloud:30367/BLUDB", echo=True)
    engine = create_engine(sqla_str+"Security=SSL;", echo=True)
    print ("Engine built")

except:
    print ("Engine failed") 

In [ ]:
# Supply connection string to engine
try:
    sa_conn = engine.connect()
    print ("SQL Alchemy connected")

except:
    print ("Unable to connect") 

# Prepare data

In [ ]:
df_fireball = pd.read_csv('../input/cneos-fireball-data/cneos_fireball_data.csv')
df_fireballtest = df_fireball.head(10)
df_fireballtest

# import data from database

In [ ]:
selectQry = "SELECT * FROM employees"

df_emp = pd.read_sql(selectQry, engine)
df_emp

# Export dataframe to database

In [ ]:
from sqlalchemy.types import Integer, Text, String, DateTime

table_name = 'emp'

df_emp.to_sql(
    table_name,
    engine,
    if_exists='replace',
    index=False,
    chunksize=500,
    dtype={
        "EMP_ID": Integer,
        "F_NAME": Text,
        "L_NAME": Text,
        "SSN": Integer,
        "B_DATE": DateTime,
        "SEX": Integer,
        "ADDRESS": Text,
        "JOB_ID": Integer,
        "SALARY": Text,
        "MANAGER_ID": Integer,
        "DEP_ID": Integer
    }
)


In [ ]:
        "EMP_ID": CHAR
        "F_NAME": VARCHAR
        "L_NAME": VARCHAR
        "SSN": CHAR
        "B_DATE": DATE
        "SEX": CHAR
        "ADDRESS": VARCHAR
        "JOB_ID": CHAR
        "SALARY": DECIMAL
        "MANAGER_ID": CHAR
        "DEP_ID": CHAR
        
        
        "job_id": Integer,
        "agency": Text,
        "business_title": Text,
        "job_category": Text,
        "salary_range_from": Integer,
        "salary_range_to": Integer,
        "salary_frequency": String(50),
        "work_location": Text,
        "division/work_unit": Text,
        "job_description": Text,
        "posting_date": DateTime,
        "posting_updated": DateTime

In [ ]:
try:
    df_emp.to_sql('emp', engine)
    print ("to_sql worked")

except:
    print ("to_sql failed") 

In [ ]:
try:
    df_fireballtest.to_sql('fireballtest1', con=sa_conn, if_exists='replace', index=False)
    print ("to_sql worked")

except:
    print ("to_sql failed") 